In [43]:
import numpy as np
from collections import defaultdict
import re

In [8]:
def keep_column():
    length = len(alignment[0])
    string_num = len(alignment)
    Keep_Column = np.ones(length).astype(int)
    Space_Count = np.zeros(length).astype(int)
    for i,string in  enumerate(alignment):
        for j,char in enumerate(string):
            if(char=="-"):
                Space_Count[j] +=1
    
    for i,count in enumerate(Space_Count):
        if(count/string_num > Theta):
            Keep_Column[i]=0
    
    return Keep_Column

In [51]:
def Initialize(length):
    Transition = defaultdict(dict)   ##hash of hash
    Emission   = defaultdict(dict)   ##hash of hash
    
    for symbol in alphabet:
        Emission['I0'][symbol]=0
    
    Transition['S']['I0'] = 0
    Transition['S']['M1'] = 0
    Transition['S']['D1'] = 0
    
    for i in range(1,length):
        
        for symbol in alphabet:
            Emission['M'+str(i)][symbol]=0
            Emission['I'+str(i)][symbol]=0
        
        Transition['I'+str(i-1)]['I'+str(i-1)] = 0
        Transition['I'+str(i-1)]['D'+str(i)] = 0
        Transition['I'+str(i-1)]['M'+str(i)] = 0
        
        Transition['D'+str(i)]['I'+str(i)] = 0
        Transition['D'+str(i)]['D'+str(i+1)] = 0
        Transition['D'+str(i)]['M'+str(i+1)] = 0
    
        Transition['M'+str(i)]['I'+str(i)] = 0
        Transition['M'+str(i)]['D'+str(i+1)] = 0
        Transition['M'+str(i)]['M'+str(i+1)] = 0
    
    
    for symbol in alphabet:
        Emission['I'+str(length)][symbol]=0
        Emission['M'+str(length)][symbol]=0
    
    Transition['I'+str(length-1)]['I'+str(length-1)] = 0
    Transition['I'+str(length-1)]['D'+str(length)] = 0
    Transition['I'+str(length-1)]['M'+str(length)] = 0
    
    Transition['I'+str(length)]['I'+str(length)] = 0
    Transition['I'+str(length)]['E']  = 0
    Transition['M'+str(length)]['I'+str(length)] = 0
    Transition['M'+str(length)]['E'] = 0
    Transition['D'+str(length)]['I'+str(length)] = 0
    Transition['D'+str(length)]['E'] = 0
    
    return Transition,Emission

In [45]:
def profile_hmm():
    string_num = len(alignment)
    ###determin which column should be removed
    Keep_Column = keep_column()   ###1 for keep, 0 for remove
    length = sum(Keep_Column)     ###length of match
    Transition,Emission = Initialize(length)
    for string in alignment:
        new_string = '' ####string excluding remove column
        for i,char in enumerate(string):
            if(Keep_Column[i]==1):
                new_string += char
        
        pre_state = 'S'        
        k = 0        
        for i,indicator in enumerate(Keep_Column):
            k += indicator
            if(indicator==1):
                if(new_string[k-1] == '-'):
                    state = 'D'+str(k)
                    Transition[pre_state][state] += 1
                    pre_state = state
                else:
                    state = 'M'+str(k)
                    Transition[pre_state][state] += 1
                    Emission[state][new_string[k-1]] += 1
                    pre_state = state
                    
            else:
                if(string[i] != '-'):
                    state = 'I'+str(k)
                    Transition[pre_state][state] += 1
                    Emission[state][string[i]] += 1
                    pre_state = state
            
        Transition[state]['E'] += 1
        
    return Transition,Emission,length

In [46]:
def Output(Transition,Emission,length):
    States = ['S','I0']
    for i in range(1,length+1):
        M = 'M'+str(i)
        States.append(M)
        D = 'D'+str(i)
        States.append(D)
        I = 'I'+str(i)
        States.append(I)
    States.append('E')
    
    
    NewTransition = defaultdict(dict)
    NewEmission   = defaultdict(dict) 
    
    for state in States:
        trans_sum = sum(Transition[state].values())
        if(trans_sum>0):
            sum_per = 0
            for state2 in States:
                if(state2 in Transition[state].keys()):
                    sum_per += Transition[state][state2]/trans_sum+sigma
                
            for state2 in States:
                if(state2 in Transition[state].keys()):
                    percent = (Transition[state][state2]/trans_sum+sigma)/sum_per
                    #ww.write('\t%s'%percent)
                    NewTransition[state][state2] = percent
                #else:
                    #ww.write('\t%d'%0)
                    #NewTransition[state][state2] = 0
        else:
            sum_per = 0
            for state2 in States:
                if(state2 in Transition[state].keys()):
                    sum_per += sigma
                
            for state2 in States: 
                if(state2 in Transition[state].keys()):
                    percent = sigma/sum_per
                    #ww.write('\t%s'%percent)
                    NewTransition[state][state2] = percent
                #else:
                    #ww.write('\t%d'%0)
                    #NewTransition[state][state2] = 0
    
    for state in States:
        if state in Emission.keys():
            emit_sum = sum(Emission[state].values())
            if(emit_sum==0):
                percent = 1/len(alphabet)
                #percent = ('%.3f' % percent)
                #percent = percent.rstrip('0').rstrip('.')
                for symbol in alphabet:    
                    #ww.write('\t%s'%percent)
                    NewEmission[state][symbol] = percent
            else:
                sum_per = 1+len(alphabet)*sigma
                for symbol in alphabet:
                    percent = (Emission[state][symbol]/emit_sum + sigma)/sum_per
                    NewEmission[state][symbol] = percent
                    #ww.write('\t%s'%percent)
        #else:
         #   for symbol in alphabet:
         #       ww.write('\t%d'%0)
    return NewTransition,NewEmission,States

In [12]:
Theta = 0.4
sigma = 0.01
alphabet = ['A', 'B', 'C', 'D', 'E','F']
alignment = ['ACDEFACADF',
'AFDA---CCF',
'A--EFD-FDC',
'ACAEF--A-C',
'ADDEFAAADF'
]

In [13]:
Transition,Emission,length = profile_hmm()

In [14]:
Output(Transition,Emission,length)

In [47]:
Theta = 0.358
sigma = 0.01
alphabet = ['A', 'B', 'C', 'D', 'E']
alignment = ['A-A',
'ADA',
'ACA',
'A-C',
'-EA',
'D-A',]

In [54]:
Transition,Emission,length = profile_hmm()
NewTransition,NewEmission,States = Output(Transition,Emission,length)

In [55]:
NewEmission

defaultdict(dict,
            {'I0': {'A': 0.2, 'B': 0.2, 'C': 0.2, 'D': 0.2, 'E': 0.2},
             'M1': {'A': 0.7714285714285715,
              'B': 0.009523809523809523,
              'C': 0.009523809523809523,
              'D': 0.2,
              'E': 0.009523809523809523},
             'I1': {'A': 0.009523809523809523,
              'B': 0.009523809523809523,
              'C': 0.326984126984127,
              'D': 0.326984126984127,
              'E': 0.326984126984127},
             'M2': {'A': 0.8031746031746032,
              'B': 0.009523809523809523,
              'C': 0.16825396825396824,
              'D': 0.009523809523809523,
              'E': 0.009523809523809523},
             'I2': {'A': 0.2, 'B': 0.2, 'C': 0.2, 'D': 0.2, 'E': 0.2}})

In [20]:
def Viterbi(States,emit_path,Transition,Emission):
    
    n = len(States)
    m = len(emit_path)
    s = defaultdict(dict)
    backtrack = np.zeros([n,m]).astype(int)
    ####Initialize
    state = 'S'
    for state2 in Transition[state]:
        if(re.match('D'),state2)
            s[state2][0] = Transition[state][state2]
        else:
            s[state2][0] = Transition[state][state2]*Emission[state2][emit_path[0]]
    i=1     
    while i < m:
        for state3 in States:
            ####ski = max all states l
            for state2 in  Transition[state]:
                if(state3 in Transition[state2]):
                    if(re.match('D'),state3):
                        s_weight = s[state][state2]*Transition[state2][state3]
                    else:
                        s_weight = s[state][state2]*Transition[state2][state3]*Emission[state3][emit_path[i]]
                    
                    if(s[state3][i]<s_weight):
                        s[state3][i] = s_weight
                        backtrack[state3][i] = state2
                
    return s,backtrack

defaultdict(dict,
            {'S': {'I0': 0.009708737864077669,
              'M1': 0.7864077669902912,
              'D1': 0.2038834951456311},
             'I0': {'I0': 0.33333333333333337,
              'M1': 0.33333333333333337,
              'D1': 0.33333333333333337},
             'M1': {'I1': 0.9901960784313726, 'E': 0.00980392156862745},
             'D1': {'I1': 0.9901960784313726, 'E': 0.00980392156862745},
             'I1': {'I1': 0.17320261437908496, 'E': 0.8267973856209151}})

In [ ]:
def Output(s,backtrack):
    n= len(States)
    m = len(emit_path)
    k = 0
    max_score = 0
    for l in range(n):
        if(s[l][m-1]>max_score):
            max_score = s[l][m-1]
            k = l
    hidden_path = States[k]
    
    for i in reversed(range(1,m)):
        k = backtrack[k][i]
        hidden_path = States[k]+hidden_path
    return hidden_path

In [42]:
s = 'I3'
import re
if re.match('D',s):
    print(s)